<a href="https://colab.research.google.com/github/melkatewabe10/Machine-learning_LST-Estimation-/blob/main/Tsm_Tn_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Tsm and Tn Prediction**

In [ ]:
!pip install rasterio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 82.5 MB/s eta 0:00:00


# **Tsm**

In [ ]:
import os
import joblib
import numpy as np
import pandas as pd
import rasterio
import matplotlib.pyplot as plt
from collections import OrderedDict
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# ===================================
# Configuration
# ===================================
data_folder = "/content/drive/MyDrive/NEW FOLDER/NEWTRANING"        # Predictor folder
model_folder = "/content/drive/MyDrive/NEW FOLDER/MODEL"     # Trained seasonal models
output_folder = "/content/drive/MyDrive/NEW FOLDER/TSM"
os.makedirs(output_folder, exist_ok=True)

years = list(range(2009, 2017))  # Adjust as needed
seasons = [f"{s:02d}" for s in range(1, 5)]  # '01' to '04'
predictor_names = ['NDVI', 'EVI', 'NDWI','LAI', 'ALB', 'ELV', 'SLP', 'DSR']

# ===================================
# Helper Function to Read Raster
# ===================================
def read_raster(raster_path):
    with rasterio.open(raster_path) as src:
        return src.read(1)

# ===================================
# Make Predictions for Seasonal Data
# ===================================
for year in years:
    for season in seasons:
        label = f"{year}_{season}"
        print(f"\nProcessing Season {label}...")

        # --------------------------
        # Step 1: Load Predictors
        # --------------------------
        predictors = OrderedDict()
        missing = False
        for var in predictor_names:
            path = os.path.join(data_folder, f"{var}_{label}.tif")
            if not os.path.exists(path):
                print(f"Missing file: {path}. Skipping {label}.")
                missing = True
                break
            predictors[var] = read_raster(path)
        if missing:
            continue

        # --------------------------
        # Step 2: Load the Corresponding Model
        # --------------------------
        model_file = os.path.join(model_folder, f"RF_{label}.pkl")
        if not os.path.exists(model_file):
            print(f"Missing model file: {model_file}. Skipping {label}.")
            continue
        model = joblib.load(model_file)
        print(f"Model loaded: {model_file}")

        # --------------------------
        # Step 3: Stack Predictors
        # --------------------------
        height, width = predictors[predictor_names[0]].shape
        feature_stack = np.vstack([predictors[var].flatten() for var in predictor_names]).T
        valid_mask = np.isfinite(feature_stack).all(axis=1)
        X_new = feature_stack[valid_mask]

        if len(X_new) < 100:
            print(f"Too few valid pixels ({len(X_new)}). Skipping {label}.")
            continue

        # --------------------------
        # Step 4: Predict with Model
        # --------------------------
        y_pred = model.predict(X_new)

        # --------------------------
        # Step 5: Prepare Metadata
        # --------------------------
        sample_raster_path = os.path.join(data_folder, f"{predictor_names[0]}_{label}.tif")
        with rasterio.open(sample_raster_path) as src_ref:
            meta = src_ref.meta.copy()
            transform = src_ref.transform
            crs = src_ref.crs
            height, width = src_ref.height, src_ref.width

        # Create a blank prediction map and fill
        prediction_map = np.full((height * width), np.nan, dtype='float32')
        prediction_map[valid_mask] = y_pred
        prediction_map = prediction_map.reshape((height, width))

        meta.update({
            "dtype": 'float32',
            "count": 1,
            "compress": "lzw",
            "transform": transform,
            "crs": crs
        })

        # --------------------------
        # Step 6: Save Prediction
        # --------------------------
        pred_file = os.path.join(output_folder, f"Tsm_{label}.tif")
        with rasterio.open(pred_file, "w", **meta) as dst:
            dst.write(prediction_map, 1)
        print(f"Prediction saved: {pred_file}")

        # --------------------------
        # Step 7: Plot the Prediction
        # --------------------------
        plt.figure(figsize=(8, 6))
        plt.imshow(prediction_map, cmap='viridis',
                   vmin=np.nanpercentile(prediction_map, 2),
                   vmax=np.nanpercentile(prediction_map, 98))
        plt.colorbar(label='Predicted LST')
        plt.title(f"LST Prediction for Season {label}")
        plt.axis('off')
        plt.tight_layout()
        plt.show()

# # --------------------------
# # Optional: Save Summary CSV
# # --------------------------
# results_list = []
# for year in years:
#     for season in seasons:
#         label = f"{year}_{season}"
#         pred_file = os.path.join(output_folder, f"Prediction_{label}.tif")
#         if os.path.exists(pred_file):
#             results_list.append({
#                 'Year': year,
#                 'Season': season,
#                 'Prediction_File': pred_file
#             })

# summary_df = pd.DataFrame(results_list)
# summary_file = os.path.join(output_folder, "Prediction_Summary.csv")
# summary_df.to_csv(summary_file, index=False)

# print("\nPrediction process completed. Summary saved to:")
# print(summary_file)


# **Tn**

In [ ]:
import os
import joblib
import numpy as np
import pandas as pd
import rasterio
import matplotlib.pyplot as plt
from collections import OrderedDict
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# ===================================
# Configuration
# ===================================
data_folder = "/content/drive/MyDrive/NEW FOLDER/NEWTRANING"        # Predictor folder
model_folder = "/content/drive/MyDrive/NEW FOLDER/MODEL"     # Trained seasonal models
output_folder = "/content/drive/MyDrive/NEW FOLDER/TN"
os.makedirs(output_folder, exist_ok=True)

years = list(range(2009, 2017))  # Adjust as needed
seasons = [f"{s:02d}" for s in range(1, 5)]  # '01' to '04'
predictor_names = ['NDVI', 'EVI', 'NDWI','LAI', 'ALB', 'ELVref', 'SLPref', 'DSRref']

# ===================================
# Helper Function to Read Raster
# ===================================
def read_raster(raster_path):
    with rasterio.open(raster_path) as src:
        return src.read(1)

# ===================================
# Make Predictions for Seasonal Data
# ===================================
for year in years:
    for season in seasons:
        label = f"{year}_{season}"
        print(f"\nProcessing Season {label}...")

        # --------------------------
        # Step 1: Load Predictors
        # --------------------------
        predictors = OrderedDict()
        missing = False
        for var in predictor_names:
            path = os.path.join(data_folder, f"{var}_{label}.tif")
            if not os.path.exists(path):
                print(f"Missing file: {path}. Skipping {label}.")
                missing = True
                break
            predictors[var] = read_raster(path)
        if missing:
            continue

        # --------------------------
        # Step 2: Load the Corresponding Model
        # --------------------------
        model_file = os.path.join(model_folder, f"RF_{label}.pkl")
        if not os.path.exists(model_file):
            print(f"Missing model file: {model_file}. Skipping {label}.")
            continue
        model = joblib.load(model_file)
        print(f"Model loaded: {model_file}")

        # --------------------------
        # Step 3: Stack Predictors
        # --------------------------
        height, width = predictors[predictor_names[0]].shape
        feature_stack = np.vstack([predictors[var].flatten() for var in predictor_names]).T
        valid_mask = np.isfinite(feature_stack).all(axis=1)
        X_new = feature_stack[valid_mask]

        if len(X_new) < 100:
            print(f"Too few valid pixels ({len(X_new)}). Skipping {label}.")
            continue

        # --------------------------
        # Step 4: Predict with Model
        # --------------------------
        y_pred = model.predict(X_new)

        # --------------------------
        # Step 5: Prepare Metadata
        # --------------------------
        sample_raster_path = os.path.join(data_folder, f"{predictor_names[0]}_{label}.tif")
        with rasterio.open(sample_raster_path) as src_ref:
            meta = src_ref.meta.copy()
            transform = src_ref.transform
            crs = src_ref.crs
            height, width = src_ref.height, src_ref.width

        # Create a blank prediction map and fill
        prediction_map = np.full((height * width), np.nan, dtype='float32')
        prediction_map[valid_mask] = y_pred
        prediction_map = prediction_map.reshape((height, width))

        meta.update({
            "dtype": 'float32',
            "count": 1,
            "compress": "lzw",
            "transform": transform,
            "crs": crs
        })

        # --------------------------
        # Step 6: Save Prediction
        # --------------------------
        pred_file = os.path.join(output_folder, f"Tn_{label}.tif")
        with rasterio.open(pred_file, "w", **meta) as dst:
            dst.write(prediction_map, 1)
        print(f"Prediction saved: {pred_file}")

        # --------------------------
        # Step 7: Plot the Prediction
        # --------------------------
        plt.figure(figsize=(8, 6))
        plt.imshow(prediction_map, cmap='viridis',
                   vmin=np.nanpercentile(prediction_map, 2),
                   vmax=np.nanpercentile(prediction_map, 98))
        plt.colorbar(label='Predicted LST')
        plt.title(f"LST Prediction for Season {label}")
        plt.axis('off')
        plt.tight_layout()
        plt.show()

# # --------------------------
# # Optional: Save Summary CSV
# # --------------------------
# results_list = []
# for year in years:
#     for season in seasons:
#         label = f"{year}_{season}"
#         pred_file = os.path.join(output_folder, f"Prediction_{label}.tif")
#         if os.path.exists(pred_file):
#             results_list.append({
#                 'Year': year,
#                 'Season': season,
#                 'Prediction_File': pred_file
#             })

# summary_df = pd.DataFrame(results_list)
# summary_file = os.path.join(output_folder, "Prediction_Summary.csv")
# summary_df.to_csv(summary_file, index=False)

# print("\nPrediction process completed. Summary saved to:")
# print(summary_file)


# Tsm not masked

In [ ]:
import os
import joblib
import numpy as np
import pandas as pd
import rasterio
import matplotlib.pyplot as plt
from collections import OrderedDict
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# ===================================
# Configuration
# ===================================
data_folder = "/content/drive/MyDrive/SEASON_II"        # Predictor folder
model_folder = "/content/drive/MyDrive/SEASON_II_MODEL"     # Trained seasonal models
output_folder = "/content/drive/MyDrive/SEASON_II_TSM"
os.makedirs(output_folder, exist_ok=True)

years = list(range(2009, 2017))  # Include 2009–2016
seasons = [f"{s:02d}" for s in range(1, 5)]  # '01', '02', '03', '04' = four seasons
predictor_names = ['NDVI', 'EVI', 'NDWI', 'ALB', 'ELV', 'SLP', 'DSR']

# ===================================
# Helper Function
# ===================================
def read_raster(raster_path):
    with rasterio.open(raster_path) as src:
        return src.read(1)

# ===================================
# Prediction Loop
# ===================================
for year in years:
    for season in seasons:
        label = f"{year}_{season}"
        print(f"\nProcessing {label}...")

        # --------------------------
        # Step 1: Load Predictors
        # --------------------------
        predictors = OrderedDict()
        missing = False
        for var in predictor_names:
            path = os.path.join(data_folder, f"{var}_{label}.tif")
            if not os.path.exists(path):
                print(f"Missing predictor file: {path}. Skipping {label}.")
                missing = True
                break
            predictors[var] = read_raster(path)
        if missing:
            continue

        # --------------------------
        # Step 2: Load Model
        # --------------------------
        model_file = os.path.join(model_folder, f"RF_{label}.pkl")
        if not os.path.exists(model_file):
            print(f"Missing model: {model_file}. Skipping {label}.")
            continue
        model = joblib.load(model_file)
        print(f"Model loaded: {model_file}")

        # --------------------------
        # Step 3: Masking with ELV as Reference
        # --------------------------
        reference_data = predictors['ELV']
        lst_mask = np.isfinite(reference_data).flatten()

        # --------------------------
        # Step 4: Stack Predictors
        # --------------------------
        height, width = predictors[predictor_names[0]].shape
        feature_stack = np.vstack([predictors[var].flatten() for var in predictor_names]).T
        valid_mask = np.isfinite(feature_stack).all(axis=1)
        combined_mask = valid_mask & lst_mask
        X_new = feature_stack[combined_mask]

        if len(X_new) < 100:
            print(f"Too few valid pixels ({len(X_new)}). Skipping {label}.")
            continue

        # --------------------------
        # Step 5: Predict
        # --------------------------
        y_pred_full = model.predict(feature_stack)

        # --------------------------
        # Step 6: Create Prediction Map
        # --------------------------
        prediction_map = np.full((height * width), np.nan, dtype='float32')
        prediction_map[lst_mask] = y_pred_full[lst_mask]
        prediction_map = prediction_map.reshape((height, width))

        # --------------------------
        # Step 7: Save as GeoTIFF
        # --------------------------
        sample_raster_path = os.path.join(data_folder, f"{predictor_names[0]}_{label}.tif")
        with rasterio.open(sample_raster_path) as src_ref:
            meta = src_ref.meta.copy()
            transform = src_ref.transform
            crs = src_ref.crs
        meta.update({
            "dtype": 'float32',
            "count": 1,
            "compress": "lzw",
            "transform": transform,
            "crs": crs
        })
        pred_file = os.path.join(output_folder, f"Tsm_{label}.tif")
        with rasterio.open(pred_file, "w", **meta) as dst:
            dst.write(prediction_map, 1)
        print(f"Prediction saved: {pred_file}")

        # --------------------------
        # Step 8: Visualization
        # --------------------------
        plt.figure(figsize=(8, 6))
        plt.imshow(prediction_map, cmap='viridis',
                   vmin=np.nanpercentile(prediction_map, 2),
                   vmax=np.nanpercentile(prediction_map, 98))
        plt.colorbar(label='Predicted LST')
        plt.title(f"LST Prediction for Season {label}")
        plt.axis('off')
        plt.tight_layout()
        plt.show()


# Tn not masked

In [ ]:
import os
import joblib
import numpy as np
import pandas as pd
import rasterio
import matplotlib.pyplot as plt
from collections import OrderedDict
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# ===================================
# Configuration
# ===================================
data_folder = "/content/drive/MyDrive/SEASON_II"        # Predictor folder
model_folder = "/content/drive/MyDrive/SEASON_II_MODEL"     # Trained seasonal models
output_folder = "/content/drive/MyDrive/SEASON_II_TN"
os.makedirs(output_folder, exist_ok=True)

years = list(range(2009, 2017))  # Include 2009–2016
seasons = [f"{s:02d}" for s in range(1, 5)]  # '01', '02', '03', '04' = four seasons
predictor_names = ['NDVI', 'EVI', 'NDWI', 'ALB', 'ELVref', 'SLPref', 'DSRref']

# ===================================
# Helper Function
# ===================================
def read_raster(raster_path):
    with rasterio.open(raster_path) as src:
        return src.read(1)

# ===================================
# Prediction Loop
# ===================================
for year in years:
    for season in seasons:
        label = f"{year}_{season}"
        print(f"\nProcessing {label}...")

        # --------------------------
        # Step 1: Load Predictors
        # --------------------------
        predictors = OrderedDict()
        missing = False
        for var in predictor_names:
            path = os.path.join(data_folder, f"{var}_{label}.tif")
            if not os.path.exists(path):
                print(f"Missing predictor file: {path}. Skipping {label}.")
                missing = True
                break
            predictors[var] = read_raster(path)
        if missing:
            continue

        # --------------------------
        # Step 2: Load Model
        # --------------------------
        model_file = os.path.join(model_folder, f"RF_{label}.pkl")
        if not os.path.exists(model_file):
            print(f"Missing model: {model_file}. Skipping {label}.")
            continue
        model = joblib.load(model_file)
        print(f"Model loaded: {model_file}")

        # --------------------------
        # Step 3: Masking with ELV as Reference
        # --------------------------
        reference_data = predictors['ELVref']
        lst_mask = np.isfinite(reference_data).flatten()

        # --------------------------
        # Step 4: Stack Predictors
        # --------------------------
        height, width = predictors[predictor_names[0]].shape
        feature_stack = np.vstack([predictors[var].flatten() for var in predictor_names]).T
        valid_mask = np.isfinite(feature_stack).all(axis=1)
        combined_mask = valid_mask & lst_mask
        X_new = feature_stack[combined_mask]

        if len(X_new) < 100:
            print(f"Too few valid pixels ({len(X_new)}). Skipping {label}.")
            continue

        # --------------------------
        # Step 5: Predict
        # --------------------------
        y_pred_full = model.predict(feature_stack)

        # --------------------------
        # Step 6: Create Prediction Map
        # --------------------------
        prediction_map = np.full((height * width), np.nan, dtype='float32')
        prediction_map[lst_mask] = y_pred_full[lst_mask]
        prediction_map = prediction_map.reshape((height, width))

        # --------------------------
        # Step 7: Save as GeoTIFF
        # --------------------------
        sample_raster_path = os.path.join(data_folder, f"{predictor_names[0]}_{label}.tif")
        with rasterio.open(sample_raster_path) as src_ref:
            meta = src_ref.meta.copy()
            transform = src_ref.transform
            crs = src_ref.crs
        meta.update({
            "dtype": 'float32',
            "count": 1,
            "compress": "lzw",
            "transform": transform,
            "crs": crs
        })
        pred_file = os.path.join(output_folder, f"Tn_{label}.tif")
        with rasterio.open(pred_file, "w", **meta) as dst:
            dst.write(prediction_map, 1)
        print(f"Prediction saved: {pred_file}")

        # --------------------------
        # Step 8: Visualization
        # --------------------------
        plt.figure(figsize=(8, 6))
        plt.imshow(prediction_map, cmap='viridis',
                   vmin=np.nanpercentile(prediction_map, 2),
                   vmax=np.nanpercentile(prediction_map, 98))
        plt.colorbar(label='Predicted LST')
        plt.title(f"LST Prediction for Season {label}")
        plt.axis('off')
        plt.tight_layout()
        plt.show()
